## What We Mean by Distributed State

If you've worked with data processing libraries like [Pandas](https://pandas.pydata.org/) or big data tools like [Apache Spark](https://spark.apache.org), you know that they provide rich features for manipulating large, structured _data sets_, i.e., the analogs of tables in a database. Some tools even support partitioning of these data sets over clusters for scalability.

This isn't the kind of distributed "state" Ray addresses. Instead, it's the more open-ended _graph of objects_ found in more general-purpose applications. For example, it could be the state of a game engine used in a reinforcement learning (RL) application or the total set of parameters in a giant neural network, some of which now have hundreds of millions of parameters.

## Conway's Game of Life

Let's explore Ray's actor model using [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway's_Game_of_Life), a famous _cellular automaton_.

Here is an example of a notable pattern of game evolution, _Gospers glider gun_: 

![Example Gospers glider gun](../images/Gospers_glider_gun.gif)

(credit: Lucas Vieira - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=101736)

We'll use an implementation of Conway's Game of Life as a nontrivial example of maintaining state, the current grid of living and dead cells. We'll see how to leverage Ray to scale it.

> **Note:** Sadly, [John Horton Conway](https://en.wikipedia.org/wiki/John_Horton_Conway), the inventor of this automaton, passed away from COVID-19 on April 11, 2020

Let's start with some imports

In [ ]:
import ray, time, sys, os
import numpy as np
import os
sys.path.append("..")         # For library helper functions
from util.printing import pd  # Printing results helper function

In [ ]:
from GameOfLife import Game, State, ConwaysRules

In [ ]:
# Utility functions for plotting using Holoviews and Bokeh, as well as running and timing games
from actor_lesson_util import new_game_of_life_graph, new_game_of_life_grid, run_games, run_ray_games

The implementation is a bit long, so all the code is contained in [`GameOfLife.py`](GameOfLife.py). We'll summarize parts of it here. 

(You can also run that file as a standalone script from the command line, try `python GameOfLife.py --help`. On MacOS and Linux machines, the script is executable; you can omit the `python`).

The first class is the `State`, which encapsulates the board state as an `N x N` grid of _cells_, where `N` is specified by the user. (For simplicity, we just use square grids.) There are two ways to initialize the game, specifying a starting grid or a size, in which case the cells are set randomly. The sample below just shows the size option. `State` instances are _immutable_, because the `Game` (discussed below) keeps a sequence of them, representing the lifetime states of the game.

For smaller grids, it's often possible that the game reaches a terminal state where it stops evolving. Larger grids are more likely to exhibit different cyclic patterns that would evolve forever, thereby making those runs appear to be _immortal_, except they eventually get disrupted by evolving neighbors. 

```python
class State:
    def __init__(self, size = 10):  
        # The version in the file also lets you pass in a grid of initial cells.
        self.size = size
        self.grid = np.random.randint(2, size = size*size).reshape((size, size))

    def living_cells(self):
        cells = [(i,j) for i in range(self.size) for j in range(self.size) if self.grid[i][j] == 1]
        return zip(*cells)
```

Next, `ConwaysRules` encapsulates the logic of computing the new state of a game from the current state, using the update rules defined as follows:

* Any live cell with fewer than two live neighbours dies, as if by underpopulation.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by overpopulation.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

This class is stateless; `step()` is passed a `State` instance and it returns a new instance for the udpated state.

```python
class ConwaysRules:
    def step(self, state):
        """
        Determine the next values for all the cells, based on the current
        state. Creates a new State with the changes.
        """
        new_grid = state.grid.copy()
        for i in range(state.size):
            for j in range(state.size):
                new_grid[i][j] = self.apply_rules(i, j, state)
        new_state = State(grid = new_grid)
        return new_state

    def apply_rules(self, i, j, state):
        # Computes the next state for grid[i][j]
```

Finally, the game holds a sequence of states and the rules "engine".

```python
class Game:
    def __init__(self, initial_state, rules):
        self.states = [initial_state]
        self.rules = rules

    def step(self, num_steps = 1):
        """Take 1 or more steps, returning a list of new states."""
        new_states = [self.rules.step(self.states[-1]) for _ in range(num_steps)]
        self.states.extend(new_states)
        return new_states
```

Okay, let's try it out!!

In [4]:
steps     =  50  # Use a larger number for a long-running game.
game_size = 100
plot_size = 800

In [5]:
def new_game(game_size):
    initial_state = State(size = game_size)
    rules = ConwaysRules()
    game  = Game(initial_state=initial_state, rules=rules)
    return game

In [6]:
game = new_game(10)
print(game.states[0])

| *         * * *     |
| * * *       * *   * |
| * * *   *   *     * |
| *       *     * * * |
|     *   *   * *     |
|   *   * *   * *     |
|         *       *   |
| * *   * * *       * |
| *   * *       *     |
|   * * *       * * * |


Now let's create a graph for a game of life using the imported utility function, `new_game_of_life_grid` (with only one graph in the "grid" for now).

**Note:** It will be empty for now.

In [7]:
_, graphs = new_game_of_life_grid(game_size, plot_size, x_grid=1, y_grid=1, shrink_factor=1.0)
graphs[0]

:DynamicMap   []
   :Points   [x,y]

To make sure we don't consume too much driver memory, since games can grow large, let's write a function, `do_trial`, to run the experiment, then when it returns, the games will go out of scope and their memory will be reclaimed. It will use a library function we imported, `run_games` and the `new_game` function above to do most of the work. 

(You might wonder why we don't create the `graphs` inside the function. It's essentially impossible to show the grid **before** the games run **and** to do the update visualization after it's shown inside one function inside a notebook cell. We have to build the grid, render it separately, then call `do_trial`.)

In [8]:
def do_trial(graphs, num_games=1, steps=steps, batch_size=1, game_size_for_each=game_size, pause_between_batches=0.0):
    games = [new_game(game_size_for_each) for _ in range(num_games)]
    run_games(games, graphs, steps, batch_size, pause_between_batches)

In [9]:
%time do_trial(graphs, pause_between_batches=0.1)

Total time for 1 games,    50 steps,     1 batch size:  9.173 seconds
CPU times: user 3.94 s, sys: 42.9 ms, total: 3.98 s
Wall time: 9.17 s


(If you can't see the plot or see it update, click [here](../images/ConwaysGameOfLife-Snapshot.png) for a screen shot.)

How much time did it take? Note that there were `steps*0.1` seconds of sleep time between steps, so the rest is compute time. Does that account for the difference between the _user_ time and the _wall_ time?

In [10]:
steps*0.1

5.0

### Running Lots of Games

Suppose we wanted to run many of these games at the same time. For example, we might use reinforcement learning to find the initial state that maximizes some _reward_, like the most live cells after `N` steps or for immortal games. You could try writing a loop that starts `M` games and run the previous step loop interleaving games. Let's try that, with smaller grids.

In [11]:
x_grid = 5
y_grid = 3
shrink_factor = y_grid  # Instead of 1 N-size game, build N/shrinkfactor size games
small_game_size = round(game_size/shrink_factor)

First build a grid of graphs, like before:

In [12]:
gridspace, all_graphs = new_game_of_life_grid(small_game_size, plot_size, x_grid, y_grid, shrink_factor)
gridspace

:GridSpace   [i,j]
   :DynamicMap   []
      :Points   [x,y]

In [13]:
%time do_trial(all_graphs, num_games=x_grid*y_grid, steps=steps, batch_size=1, game_size_for_each=small_game_size, pause_between_batches=0.1)

Total time for 15 games,    50 steps,     1 batch size: 21.042 seconds
CPU times: user 15.5 s, sys: 386 ms, total: 15.9 s
Wall time: 21 s


(If you can't see the plot in the previous cell output, click [here](../images/ConwaysGameOfLife-Grid-Snapshot.png).)

How much time did it take?  You can perceive a "wave" across the graphs at each time step, because the games aren't running at the same time, then a bunch of update spurts happen periodically, etc. Not so great...

There were the same `steps*0.1` seconds of sleep time between steps, not dependent on the number of games, so the rest is compute time.

Let's explore improving the performance with Ray.

In [14]:
ray.init(ignore_reinit_error=True)

2020-04-23 12:08:33,704	INFO resource_spec.py:212 -- Starting Ray with 4.2 GiB memory available for workers and up to 2.12 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-23 12:08:34,078	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:32896',
 'object_store_address': '/tmp/ray/session_2020-04-23_12-08-33_694416_43518/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-23_12-08-33_694416_43518/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-23_12-08-33_694416_43518'}

In [15]:
print(f'New port? http://{ray.get_webui_url()}')

New port? http://localhost:8265


## Actors - Ray's Tool for Distributed State

Python is an object-oriented language. We often encapsulate bits of state in classes, like we did for `State` above. Ray leverages this familiar mechanism to manage distributed state.

Recall that adding the `@ray.remote` annotation to a _function_ turned it into a _task_. If we use the same annotation on a Python _class_, we get an _actor_.

### Why "Actor"

The [Actor Model of Concurrency](https://en.wikipedia.org/wiki/Actor_model) is almost 50 years old! It's a _message-passing_ model, where autonomous blocks of code, the actors, receive messages from other actors asking them to perform work or return some results. Implementations provide thread safety while the messages are processed, one at a time. This means the user of an actor model implementation doesn't have to worry about writing thread-safe code. Because many messages might arrive while one is being processed, they are stored in a queue and processed one at a time, the order of arrival. 

There are many other implementations of the actor model, including [Erlang](https://www.erlang.org/), the first system to create a production-grade implementation, initially used for telecom switches, and [Akka](https://akka.io), a JVM implementation inspired by Erlang.

Let's start by simply making `Game` an actor. We'll just subclass it and add `@ray.remote` to the subclass.

There's one other change we have to make; if we want to access the `state` and `rules` instances in an Actor, we can't just use `mygame.state`, for example, as you would normally do for Python instances. Instead, we have to add "getter" methods for them.

Here's our Game actor definition.

In [16]:
@ray.remote
class RayGame(Game):
    def __init__(self, initial_state, rules):
        super().__init__(initial_state, rules)
        
    def get_states(self):
        return self.states
            
    def get_rules(self):
        return self.rules

To construct an instance and call methods, you use `.remote` as for tasks:

In [17]:
def new_ray_game(game_size):
    initial_state = State(size = game_size)
    rules = ConwaysRules()
    ray_game_actor = RayGame.remote(initial_state, rules)   # Note that .remote(...) is used to construct the instance.
    return ray_game_actor

We'll use the following function to try out the implementation, but then take the Ray actor out of scope when we're done. This is because actors remain pinned to a worker as long as the driver (this notebook) has a reference to them. We don't want that wasted space...

In [18]:
def try_ray_game_actor():
    ray_game_actor = new_ray_game(small_game_size)
    print(f'Actor for game: {ray_game_actor}')
    init_states = ray.get(ray_game_actor.step.remote())
    print(f'Initial state:\n{init_states[0]}')
    new_states = ray.get(ray_game_actor.step.remote())
    print(f'State after step #1:\n{new_states[0]}')
try_ray_game_actor()

Actor for game: Actor(RayGame, 45b95b1c0100)
Initial state:
| *   * *   *   * * * * * * *               * * * * * *         *   |
| *   *   *         * * * *   * *                 *         *       |
|             * *     * * * *   *                 *   *   * *       |
| *       *   *   *         * * *                       * *         |
|           * *   * *                             *             * * |
|           * *   *           *                 *     * *           |
| *       * *     *       *   * * *               * * *             |
| * * *                     * *   *   * *   * * * *   *             |
| * *       *   *         *       * *       * *   * *               |
| *   *     * * *         *               *       * *   *           |
|           *             *                     * *   *             |
| * * * *   *             *       *   *             * *           * |
| *     *             * *       * * *             * * *         * * |
| * *   * *       * *   * * * 

> **Key Points:** To summarize:
>
> 1. Declare an _actor_ by annotating a class with `@ray.remote`, just like declaring a _task_ from a function.
> 2. Add "getter" methods for any data members that you need to access, because direct access, such as `my_game.state`, doesn't work for actors.
> 3. Construct actor instances with `my_instance = MyClass.remote(...)`.
> 4. Call methods with `my_instance.some_method.remote(...)`.
> 5. Use `ray.get()` and `ray.wait()` to retrieve results, just like you do for task results.

> **Tip:** If you start getting warnings about lots of Python processes running or you have too many actors scheduled. use `ray.kill(actor_id)` to remove the ones we no longer need. For example, to kill the just-created actor, `ray.kill(raygame_id)`. We'll see more examples later.

Okay, now let's repeat our grid experiment with a Ray-enabled Game of Life. Let's define a helper function, `do_ray_trail`, which is analogous to `do_trial` above. It encapsulates some of the steps, for the same reasons mentioned above; so that our actors go out of scope and the worker slots are reclaimed when the function call returns.

We call a library function `run_ray_games` to run these games. It's somewhat complicated, because it uses `ray.wait()` to process updates as soon as they are available, and also has hooks for batch processing and running without graphing (see below).

We'll create the graphs separately and pass them into `do_ray_trial`. 

In [19]:
def do_ray_trial(graphs, num_games=1, steps=steps, batch_size=1, game_size_for_each=game_size, pause_between_batches=0.0):
    game_actors = [new_ray_game(game_size_for_each) for _ in range(num_games)]
    run_ray_games(game_actors, graphs, steps, batch_size, pause_between_batches)

In [20]:
ray_gridspace, ray_graphs = new_game_of_life_grid(small_game_size, plot_size, x_grid, y_grid, shrink_factor)
ray_gridspace

:GridSpace   [i,j]
   :DynamicMap   []
      :Points   [x,y]

In [ ]:
%time do_ray_trial(ray_graphs, num_games=x_grid*y_grid, steps=steps, batch_size=1, game_size_for_each=small_game_size, pause_between_batches=0.1)

2020-04-23 12:33:30,164	WARNING worker.py:1072 -- A worker died or was killed while executing task ffffffffffffffffd7e9a6920100.


(pid=44126) *** Aborted at 1587670410 (unix time) try "date -d @1587670410" if you are using GNU date ***
(pid=44126) PC: @                0x0 (unknown)
(pid=44126) *** SIGSEGV (@0x80) received by PID 44126 (TID 0x700007848000) stack trace: ***
(pid=44126)     @     0x7fff67a9442d _sigtramp
(pid=44126)     @                0x0 (unknown)
(pid=44126)     @        0x10909a819 ray::TaskManager::NumPendingTasks()
(pid=44126)     @        0x109035b06 ray::CoreWorker::HandleGetCoreWorkerStats()
(pid=44126)     @        0x1090516a2 ray::rpc::ServerCallImpl<>::HandleRequestImpl()
(pid=44126)     @        0x1090515e4 _ZN5boost4asio6detail18completion_handlerIZN3ray3rpc14ServerCallImplINS4_24CoreWorkerServiceHandlerENS4_25GetCoreWorkerStatsRequestENS4_23GetCoreWorkerStatsReplyEE13HandleRequestEvEUlvE_E11do_completeEPvPNS1_19scheduler_operationERKNS_6system10error_codeEm
(pid=44126)     @        0x109607ea3 boost::asio::detail::scheduler::do_run_one()
(pid=44126)     @        0x1095fbce2 boost::as

How did your times compare? On a MacBook Pro 13", late 2019 model, this run took roughly 19 seconds vs. 21 seconds for the previous run without Ray, which isn't a huge improvement. 

In fact, updating the graphs causes enough overhead to remove most of the speed advantage of using Ray. However, it's much nicer having smoother graph updates (at least after a few seconds).

So, if we want to study more performance optimizations, we should remove the graphing overhead, which we'll do for the rest of this lesson.

Let's run the two trials without graphs and compare the performance. We'll use no pauses between "batches" and run the same number of games as the number of CPU (cores) Ray says we have. This is actually the number of workers Ray started for us and 2x the number of actual cores:

In [34]:
num_cpus_float = ray.cluster_resources()['CPU']
num_cpus_float

8.0

As soon as you start the next two cell, switch to the Ray Dashboard and watch the CPU utilization. You'll see the Ray workers are idle, but the total CPU utilization will be about 20-25%, for a four-core machine. We're running completely in the Python process for this notebook, which only utilizes one core.

In [36]:
%time do_trial(None, num_games=round(num_cpus_float), steps=steps, batch_size=1, game_size_for_each=game_size, pause_between_batches=0.0)

Total time for 8 games,    50 steps,     1 batch size: 25.414 seconds
CPU times: user 24.8 s, sys: 457 ms, total: 25.2 s
Wall time: 25.4 s


Similarly, as soon as you start the next two cell, switch to the Ray Dashboard and watch the CPU utilization. Now, the Ray workers will be utilized (but not 100%) and the total CPU utilization will be about 70%. Now we're running on all cores.

In [37]:
%time do_ray_trial(None, num_games=round(num_cpus_float), steps=steps, batch_size=1, game_size_for_each=game_size, pause_between_batches=0.0)

Total time for 8 games,    50 steps,     1 batch size: 10.531 seconds
CPU times: user 2.78 s, sys: 139 ms, total: 2.92 s
Wall time: 10.5 s


So, running parallel games does benefit from Ray. The performance boost is about 2x, which isn't especially large running on a laptop, because the computation is CPU intensive for each game with frequent memory access. We would see much more impressive improvements on a cluster when running a massive number of games.

Notice the times for `user` and `total` times for each case. These lines are the outputs of the `%time` "magic" and they are only measuring the time for the notebook Python process (i.e., our "driver" program.) Without Ray, all the work is done in this process, so the times roughtly equal the wall clock time. However, for Ray, these times are very low; the notebook is mostly idle, while the work is done in the worker processes.

## More about Actors

Let's finish with a discussion of additional important information about actors. Some of the points were mentioned above.

### Actor Scheduling and Lifetimes

For the most part, when Ray runs actor code, it uses the same _task_ mechanisms we discussed in _Task Parallelism, Parts 1 and 2_. Actor constructor and method invocations work just like task invocations. However, there are a few notable differences:

* However, once a task finishes, it is removed from the worker that executed it, while an actor is _pinned_ to the worker until all Python references to it are out of scope. That is, the usual garbage collection mechanism in Python determines when an actor is removed from a worker. The reason the actor must remain in memory is because it holds state that might be needed, whereas tasks are stateless.
* Currently, each actor instance uses tens of MB of memory overhead. Hence, just as you should avoid having too many fine-grained tasks, you should avoid too many actor instances. (Reducing the overhead per actor is an ongoing improvement project.)

### Durability of Actor State

At this time, Ray provides no built-in mechanism for persisting actor state. Hence, if a worker or whole server goes down with actor instances, their state is lost. 

Actor state is stored in the object store, just like the results of task executions. Even if a worker goes down but the object store is healthy, the reference to the object is lost.

This is an area where Ray will evolve and improve in the future. For now, an important design consideration is to decide when you need to _checkpoint_ state and to use an appropriate mechanism for this purpose. Some of the Ray APIs explored in other tutorials have built-in checkpoint features, such as for saving snapshots of trained models to a file system.

## Extra - Does It Help to Run with Larger Batch Sizes?

You'll notice that we defined `run_games` and `do_trial`, as well as `run_ray_games` and `do_ray_trial` to take an optional `batch_size` that defaults to `1`. The idea is that maybe running game steps in batches, rather than one step at a time, will improve performance (but look less pleasing in the graphs). 

This concept works in some contexts, such as minimizing the number of messages sent in networks (that is, fewer, but larger payloads), but it actually doesn't help a lot here, because each game is played in a single process, whether using Ray or not (at least as currently implemented...). Batching reduces the number of method invocations, but it's not an important amount of overhead in our case.

Let's confirm our suspicion about batching, that it doesn't help a lot.

Let's time several batch sizes without and with Ray. We'll run several times with each batch size to get an informal sense of the variation possible:

In [35]:
for batch in [1, 10, 25, 50]:
    for run in [0, 1]:
        do_trial(graphs = None, num_games=1, steps=steps, batch_size=batch, game_size_for_each=game_size, pause_between_batches=0.0)

Total time for 1 games,    50 steps,     1 batch size:  3.171 seconds
Total time for 1 games,    50 steps,     1 batch size:  3.218 seconds
Total time for 1 games,    50 steps,    10 batch size:  3.516 seconds
Total time for 1 games,    50 steps,    10 batch size:  3.146 seconds
Total time for 1 games,    50 steps,    25 batch size:  3.174 seconds
Total time for 1 games,    50 steps,    25 batch size:  3.224 seconds
Total time for 1 games,    50 steps,    50 batch size:  3.203 seconds
Total time for 1 games,    50 steps,    50 batch size:  3.198 seconds


There isn't a significant difference based on batch size.

What about Ray? If we're running just one game, the results should be about the same.

In [41]:
for batch in [1, 10, 25, 50]:
    for run in [0, 1]:
        do_ray_trial(graphs = None, num_games=1, steps=steps, batch_size=batch, game_size_for_each=game_size, pause_between_batches=0.0)

2020-04-23 11:52:58,079	WARNING worker.py:1072 -- The actor or task with ID ffffffffffffffff714ae4f70100 is pending and cannot currently be scheduled. It requires {} for execution and {CPU: 1.000000} for placement, but this node only has remaining {CPU: 8.000000}, {object_store_memory: 1.513672 GiB}, {node:192.168.1.149: 1.000000}, {memory: 4.443359 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.


Total time for 1 games,    50 steps,     1 batch size:  3.445 seconds
Total time for 1 games,    50 steps,     1 batch size:  3.310 seconds
Total time for 1 games,    50 steps,    10 batch size:  3.245 seconds
Total time for 1 games,    50 steps,    10 batch size:  3.256 seconds
Total time for 1 games,    50 steps,    25 batch size:  3.161 seconds
Total time for 1 games,    50 steps,    25 batch size:  3.271 seconds
Total time for 1 games,    50 steps,    50 batch size:  3.198 seconds
Total time for 1 games,    50 steps,    50 batch size:  3.216 seconds


With Ray's background activity, there is likely to be a little more variation in the numbers, but the conclusion is the same; the batch size doesn't matter because no additional exploitation of asynchronous computing is used.